In [4]:
import random
from pyspark.sql import SparkSession
from faker import Faker
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("LazyEvaluation").getOrCreate()

24/05/25 19:48:20 WARN Utils: Your hostname, paulo resolves to a loopback address: 127.0.1.1; using 172.27.58.218 instead (on interface eth0)
24/05/25 19:48:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/25 19:48:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
fake = Faker()
num_records = 1000
activities = ("running", "swimming", "reading", "cooking", "gaming", "reading")
for _ in range(num_records):
    fake_data = pd.DataFrame(
        {
            "user_id": [fake.uuid4() for _ in range(num_records)],
            "activity": [
                fake.random_element(elements=activities) for _ in range(num_records)
            ],
            "start_timestamp": [
                fake.date_time_this_year().timestamp() for _ in range(num_records)
            ],
            "end_timestamp": [
                fake.date_time_this_year().timestamp() + random.randint(0, 86400)
                for _ in range(num_records)
            ],
            "location": [fake.city() for _ in range(num_records)],
            "age": [fake.random_int(min=18, max=80) for _ in range(num_records)],
        }
    )

# Convert Pandas DataFrame to PySpark DataFrame
user_activity_df = spark.createDataFrame(fake_data)

user_activity_df.write.parquet("tmp/data/user_activity", mode="overwrite")
user_activity_df = spark.read.parquet("tmp/data/user_activity")

24/05/25 19:56:00 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
24/05/25 19:56:00 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
24/05/25 19:56:02 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [ ]:
# Load the user activity data
user_activity_df = spark.read.parquet("tmp/data/user_activity")
user_activity_df.show(5)

+--------------------+--------+-------------------+-------------------+-------------------+---+
|             user_id|activity|    start_timestamp|      end_timestamp|           location|age|
+--------------------+--------+-------------------+-------------------+-------------------+---+
|24b142ce-0f7e-449...| reading|1.707829383915254E9|1.712826204423252E9|   West Ronaldmouth| 38|
|52e12b9c-b910-491...| reading|1.707039871804524E9|1.710571944809592E9|  South Richardbury| 65|
|fb52fdb6-0dc6-420...|swimming|1.704138366546621E9| 1.70540251114133E9|West Stephenchester| 26|
|1d83172a-a3b9-41d...|  gaming|1.708359939943264E9|1.711729919060253E9|     West Nathaniel| 48|
|3e5ce184-52cd-4e5...|swimming|1.706404599417025E9| 1.71497861549062E9|  East Jeffreyville| 42|
+--------------------+--------+-------------------+-------------------+-------------------+---+
only showing top 5 rows



In [ ]:
# Define transformations
df_calculated = user_activity_df.withColumn(
    "activity_duration",
    user_activity_df["end_timestamp"] - user_activity_df["start_timestamp"],
)
df_filtered = df_calculated.filter(df_calculated["age"] > 21)
df_selected = df_filtered.select("user_id", "activity_duration")
df_result = df_selected.groupBy("user_id").sum("activity_duration")

In [ ]:
df_result.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[user_id#375], functions=[sum(activity_duration#413)])
   +- Exchange hashpartitioning(user_id#375, 200), ENSURE_REQUIREMENTS, [plan_id=222]
      +- HashAggregate(keys=[user_id#375], functions=[partial_sum(activity_duration#413)])
         +- Project [user_id#375, (end_timestamp#378 - start_timestamp#377) AS activity_duration#413]
            +- Filter (isnotnull(age#380L) AND (age#380L > 21))
               +- FileScan parquet [user_id#375,start_timestamp#377,end_timestamp#378,age#380L] Batched: true, DataFilters: [isnotnull(age#380L), (age#380L > 21)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/mnt/c/Users/paulo_ufva11y/OneDrive/Documentos/Estudos/data-engin..., PartitionFilters: [], PushedFilters: [IsNotNull(age), GreaterThan(age,21)], ReadSchema: struct<user_id:string,start_timestamp:double,end_timestamp:double,age:bigint>


